<a href="https://colab.research.google.com/github/sharathc21/ML/blob/main/AmzonReviewLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer as snow
import sqlite3

In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')


In [4]:
# !mkdir  kaggle  # create directory kaggle

In [5]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sharathc21","key":"bb510ccddd29f2287c08ac34773e4f07"}'}

In [6]:
!ls

kaggle.json  sample_data


In [7]:
!mkdir ~/.kaggle
!cp "kaggle.json" ~/.kaggle/

In [8]:
# !cp "kaggle.json" ~/kaggle.json

In [9]:
!kaggle datasets download -d snap/amazon-fine-food-reviews



 93% 225M/242M [00:02<00:00, 123MB/s]
100% 242M/242M [00:02<00:00, 120MB/s]


In [10]:
!unzip amazon-fine-food-reviews.zip

Archive:  amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [11]:
conn1= sqlite3.connect('database.sqlite')


In [12]:
data= pd.read_sql_query('''
SELECT * FROM Reviews WHERE Score!=3
''',conn1)

In [13]:
data.shape

(525814, 10)

In [14]:
# data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Reviews.csv')

In [15]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = data['Score']

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1


In [16]:

positiveNegative = actualScore.map(partition) 
data['Score'] = positiveNegative
print("Number of data points in our data", data.shape)
data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [17]:
data.shape

(525814, 10)

In [18]:
# display = pd.read_sql_query("""
# SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
# FROM Reviews
# GROUP BY UserId
# HAVING COUNT(*)>1
# """, conn1)

In [19]:

data['Score']=data['Score'].map(lambda x:'Positive' if x>3 else 'Negative')
sorteddata= data.sort_values('ProductId',axis=0)


In [20]:
finaldata= sorteddata.drop_duplicates(subset={'UserId','ProfileName',\
        'Time','Text'}, keep='first',inplace=False)


In [21]:
# Removing not helpful reviews
finaldata= finaldata[finaldata['HelpfulnessNumerator'] <= finaldata['HelpfulnessDenominator']]
data= finaldata.sort_values('Time',axis=0)
data.shape

(364171, 10)

In [22]:
# Cleaning HTML, punctuations, apply stemming, lowercasing etc without removing stopwords


def cleanhtml(sentance): #substitute expression contained in <> with ' '
    cleaned= re.sub(re.compile('<.*?>'),' ',sentance)
    return cleaned
#function for removing punctuations chars
def cleanpunc(sentance):
    cleaned= re.sub(r'[?|!|\'|"|#]',r'',sentance)
    cleaned= re.sub(r'[.|,|)|(|\|/]',r'',sentance)
    return cleaned
snowstem= snow('english')

In [23]:
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
for sent in data['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        # we have used cleanpunc(w).split(), one more split function here 
        # because consider w="abc.def", cleanpunc(w) will return "abc def"
        # if we dont use .split() function then we will be considring "abc def" 
        # as a single word, but if you use .split() function we will get "abc", "def"
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(snowstem.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                if(data['Score'].values)[i] =='Negative':
                    all_negative_words.append(s)
            else:
                continue
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)


In [24]:
import tqdm

In [25]:
# storing data till now
data['CleanedText']=final_string 
#adding a column of CleanedText which displays the data after pre-processing of the review 
data['CleanedText']=data['CleanedText'].str.decode("utf-8")
    # store final table into an SQlLite table for future.
conn2 = sqlite3.connect('cleanedTextData.sqlite')
c=conn2.cursor()
conn2.text_factory = str
data.to_sql('Reviews', conn2,  schema=None, if_exists='replace', \
        index=True, index_label=None, chunksize=None, dtype=None)
conn2.close()
     

In [26]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Negative,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witti littl book make son laugh loud reci...
138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,Negative,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,can rememb see the show when air televis year ...
417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,Negative,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,beetlejuic well written movi everyth about fro...
346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,Negative,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",twist rumplestiskin captur film star michael k...
417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,Negative,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,beetlejuic excel and funni movi keaton hilari ...


In [27]:
Data= data[:100000]
Data= Data[['CleanedText','Score']]
Data['Score']= Data['Score'].map(lambda x:1 if x=='Positive' else 0)

Data_x= Data['CleanedText']
Data_y= Data['Score']

In [28]:
Data_x.index= [i for i in range(0,10**5)]
Data_x

0        this witti littl book make son laugh loud reci...
1        can rememb see the show when air televis year ...
2        beetlejuic well written movi everyth about fro...
3        twist rumplestiskin captur film star michael k...
4        beetlejuic excel and funni movi keaton hilari ...
                               ...                        
99995    this veri tasti protein shake and give you nic...
99996    this the best tast oliv and healthi for you to...
99997    sack melitta coffe label fine grind blanc noir...
99998    final babi food that tast love that great prot...
99999    first ventur salt and happi bought this and sa...
Name: CleanedText, Length: 100000, dtype: object

In [29]:
# Making Vocabulary set and Frequency dictionary of words

list_= []
for i in Data_x:
    list_ += i
list_= ''.join(list_)
allWords=list_.split()

In [30]:
# vocabulary= set(allWords)

In [31]:
vocabulary_list= list(set(allWords))

In [32]:
len(vocabulary_list)

102782

In [33]:
len(allWords)

5315502

In [34]:
# #frequency dictionary
# freq_dict= {}
# for word in vocabulary_list:
#     freq_dict[word]= allWords.count(word)

In [35]:
from tqdm import tqdm

#frequency dictionary
freq_dict = {}
for word in tqdm(vocabulary_list):
    freq_dict[word] = allWords.count(word)


100%|██████████| 102782/102782 [2:50:45<00:00, 10.03it/s]


In [37]:
Data_x[1]

'can rememb see the show when air televis year ago when was child sister later bought the which have this day thirti somethingi use this seri book song when did student teach for preschool turn the whole school now purchas along with the book for children the tradit live'

In [4]:
# import pickle
# with open('freq_dict.pkl','wb') as file:
#     pickle.dump(freq_dict,file)

In [8]:
# import pickle
# with open('freq_dict.pkl','rb') as file:
#   freq_dict=pickle.load(file)

In [41]:
from operator import itemgetter
sorted_list= []
for k, v in sorted(freq_dict.items(), key=itemgetter(1),reverse=True):
    sorted_list.append(k)

In [42]:
top_words= 5000
sorted_list= sorted_list[:5000]

In [43]:
# Transforming Sentences of words to sequence of rank number of words

column=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(word)
    column.append(' '.join(lis))

In [44]:
with open('column.pkl','wb') as file:
    pickle.dump(column,file)

In [45]:
final_x=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(sorted_list.index(word)+1)
    final_x.append(lis)

In [46]:
len(final_x)

100000

In [47]:
Xtest= final_x[:30000]
Ytest= Data_y[:30000]
Xtrain= final_x[30000:]
Ytrain= Data_y[30000:]

In [48]:
print(len(Xtrain[1]))
print(Xtrain[1])

57
[8, 216, 209, 1980, 106, 8, 368, 25, 3984, 528, 3217, 3217, 721, 17, 488, 1101, 2, 583, 127, 5, 1071, 474, 5, 11, 698, 8, 47, 28, 1, 557, 16, 3217, 3217, 9, 322, 1, 31, 89, 100, 12, 10, 156, 622, 42, 91, 151, 69, 541, 35, 1133, 17, 61, 5, 8, 352, 1, 69]


In [52]:
# from keras.preprocessing import sequence
# import keras
from keras.utils import pad_sequences
max_review_length=600
Xtrain = pad_sequences(Xtrain, maxlen=max_review_length)
Xtest= pad_sequences(Xtest, maxlen=max_review_length)

In [53]:
print(Xtrain[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [54]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6746639458186579872
xla_global_id: -1
]


In [56]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)


In [72]:
embedding_vecor_length = 32
model1 = Sequential()
model1.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 600, 32)           160032    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213,333
Trainable params: 213,333
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history1=  model1.fit(Xtrain, Ytrain,
          batch_size=64,
          epochs=2,
          verbose=1,
          validation_data=(Xtest, Ytest))# Final evaluation of the model
scores = model1.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
1094/1094 [==============================] - ETA: 0s - loss: 0.0057 - accuracy: 0.9994

In [9]:

# import pickle
# with open('LSTM_model1.pkl','wb') as file:
#    pickle.dump(model1, file)


In [61]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time
# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334
# this function is used to update the plots for each epoch and error
def plt_dynamic(x, vy, ty, ax, colors=['b']):
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

In [63]:
score= model.evaluate(Xtest, Ytest, verbose=0)
print('Test score: ',score[0])
print('Test accuracy: ',score[1])

Test score:  1.1144845757371513e-06
Test accuracy:  1.0


In [67]:
fig,ax = plt.subplots(1,1)


<IPython.core.display.Javascript object>

In [68]:
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')
x = list(range(1,11))

In [10]:
# vy = history.history['val_loss']
# ty = history.history['loss']